<a href="https://colab.research.google.com/github/Joycechidi/Secure-and-Private-AI/blob/master/Federated_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Section: Federated Learning
# Lesson: Introducing Federated Learning
Federated Learning is a technique for training Deep Learning models on data to which you do not have access. Basically:
Federated Learning: Instead of bringing all the data to one machine and training a model, we bring the model to the data, train it locally, and merely upload "model updates" to a central server.

Use Cases:
- app company (Texting prediction app)
- predictive maintenance (automobiles / industrial engines)
- wearable medical devices
- ad blockers / autotomplete in browsers (Firefox/Brave)


Challenge Description: 
- data is distributed amongst sources but we cannot aggregated it because of:
- privacy concerns: legal, user discomfort, competitive dynamics
- engineering: the bandwidth/storage requirements of aggregating the larger dataset


In [0]:
!pip install syft

     |████████████████████████████████| 215kB 2.9MB/s 
     |████████████████████████████████| 389kB 46.2MB/s 
     |████████████████████████████████| 1.4MB 42.2MB/s 
     |████████████████████████████████| 256kB 48.7MB/s 
     |████████████████████████████████| 71kB 10.6MB/s 
     |████████████████████████████████| 204kB 49.0MB/s 
     |████████████████████████████████| 450kB 45.7MB/s 
     |████████████████████████████████| 276kB 42.9MB/s 
     |████████████████████████████████| 51kB 18.2MB/s 
     |████████████████████████████████| 122kB 48.4MB/s 
  Stored in directory: /root/.cache/pip/wheels/ad/9a/f4/3105b5209674ac77fcca7fede95184c62a95df0196888e0e76
  Stored in directory: /root/.cache/pip/wheels/16/27/a1/775c62ddea7bfa62324fd1f65847ed31c55dadb6051481ba3f
Successfully built zstd pyyaml
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Found existing installation: msgpack 0.5.6
    Uninstalling msgpack-0.5.6:
     

In [0]:
%config IPCompleter.greedy=True

In [0]:
import torch as th
import syft as sy

In [0]:
hook = sy.TorchHook(th)

In [0]:
chidi = sy.VirtualWorker(hook, id="chidi")# Virtual worker simulates the interface we might have.

In [0]:
hook = sy.TorchHook(th)

W0629 16:09:55.569190 140438201272192 hook.py:98] Torch was already hooked... skipping hooking process


In [0]:
chidi._objects

{}

In [0]:
x = th.tensor([1, 2, 3, 4, 5])

In [0]:
x = x.send(chidi)

In [0]:
chidi._objects

{98851124052: tensor([1, 2, 3, 4, 5])}

In [0]:
x.location == chidi

True

In [0]:
x.location

<VirtualWorker id:chidi #objects:1>

In [0]:
x.id_at_location

98851124052

In [0]:
x.id

37634737044

In [0]:
x.owner

<VirtualWorker id:me #objects:0>

In [0]:
hook.local_worker

<VirtualWorker id:me #objects:0>

In [0]:
x

(Wrapper)>[PointerTensor | me:37634737044 -> chidi:98851124052]

In [0]:
x = x.get()

In [0]:
x

tensor([1, 2, 3, 4, 5])

# Project: # **Playing With Remote Tensors**

Use .send() and .get() atensor toTWO workers by calling .send(chidi, ify). This will require the creation of another Virtual Worker called ify.

In [0]:
#chidi = sy.VirtualWorker(hook, id="chidi")
# Virtual worker simulates the interface we might have.
ify = sy.VirtualWorker(hook, id="ify")# Virtual worker simulates the interface we might have.

In [0]:
hook = sy.TorchHook(th)

W0629 16:10:33.503199 140438201272192 hook.py:98] Torch was already hooked... skipping hooking process


In [0]:
x = th.tensor([2, 4, 6, 8, 10])
#y = th.tensor([5, 10, 15, 20, 25]).send(ify)

In [0]:
xz = x.send(chidi, ify)

In [0]:
xz.get()

[tensor([ 2,  4,  6,  8, 10]), tensor([ 2,  4,  6,  8, 10])]

In [0]:
x = th.tensor([2, 4, 6, 8, 10]).send(chidi, ify)

In [0]:
x.get(sum_results=True)

tensor([ 4,  8, 12, 16, 20])

# Remote Arithmetic

In [0]:
x = th.tensor([2, 4, 6, 8, 10]).send(chidi)
y = th.tensor([5, 10, 15, 20, 25]).send(chidi)

In [0]:
x

(Wrapper)>[PointerTensor | me:24460121927 -> chidi:12057123932]

In [0]:
y

(Wrapper)>[PointerTensor | me:3881474247 -> chidi:69593932750]

In [0]:
z = x + y

In [0]:
z

(Wrapper)>[PointerTensor | me:18828726270 -> chidi:28817841801]

In [0]:
z = z.get()
z

tensor([ 7, 14, 21, 28, 35])

In [0]:
z = th.add(x, y)
z

(Wrapper)>[PointerTensor | me:11878880186 -> chidi:11529413260]

In [0]:
x = th.tensor([2., 4, 6, 8, 10], requires_grad=True).send(chidi)
y = th.tensor([5., 10, 15, 10, 25], requires_grad=True).send(chidi)


In [0]:
z = (x + y).sum()

In [0]:
z.backward()

(Wrapper)>[PointerTensor | me:33701809959 -> chidi:27567273073]

In [0]:
x = x.get()

In [0]:
x

tensor([ 2.,  4.,  6.,  8., 10.], requires_grad=True)

In [0]:
x.grad

tensor([1., 1., 1., 1., 1.])

### Project: Learn a Simple Linear Model

 I'd like for you to create a simple linear model which will solve for the following dataset below. You should use only Variables and .backward() to do so (no optimizers or nn.Modules). Furthermore, you must do so with both the data and the model being located on Bob's machine.

In [0]:
input = th.tensor([[2.,1],[0,2,],[1,0],[0,0]], requires_grad=True).send(chidi)
target = th.tensor([[1.],[1],[0],[0]], requires_grad=True).send(chidi)

In [0]:
weights = th.tensor([[0.], [0.]], requires_grad=True).send(chidi)

In [0]:
for i in range(10):
    pred = input.mm(weights)
    
    loss = ((pred - target)**2).sum()
    
    loss.backward()
    
    weights.data.sub_(weights.grad * 0.1)
    weights.grad *= 0
    
    print(loss.get().data)

tensor(2.)
tensor(0.3600)
tensor(0.0976)
tensor(0.0556)
tensor(0.0489)
tensor(0.0478)
tensor(0.0477)
tensor(0.0476)
tensor(0.0476)
tensor(0.0476)


# **Toy Federated Learning Project**

Start by training a toy model using the centralized way. 


*   a dataset
*   a model
some basic training logic for training a model to fit the data.



In [0]:
from torch import nn, optim

In [0]:
#Using a toy dataset
data = th.tensor([[1.,1],[0,1],[1,0],[0,0]], requires_grad=True)
target = th.tensor([[1.],[1], [0], [0]], requires_grad=True)


In [0]:
#A Toy Model
model = nn.Linear(2, 1)

In [0]:
opt = optim.SGD(params=model.parameters(), lr=0.1)

In [0]:
def train(iterations=20):
    for iter in range (iterations):
        opt.zero_grad()
        
        pred = model(data)
        
        loss = ((pred - target)**2).sum()
        
        loss.backward()
        
        opt.step()
        
        print(loss.data)
        
train()

tensor(1.2749)
tensor(0.4591)
tensor(0.2721)
tensor(0.1765)
tensor(0.1161)
tensor(0.0769)
tensor(0.0512)
tensor(0.0342)
tensor(0.0231)
tensor(0.0156)
tensor(0.0107)
tensor(0.0073)
tensor(0.0051)
tensor(0.0036)
tensor(0.0025)
tensor(0.0018)
tensor(0.0013)
tensor(0.0009)
tensor(0.0007)
tensor(0.0005)


In [0]:
data_chidi = data[0:2].send(chidi)
target_chidi = target[0:2].send(chidi)

In [0]:
data_ify = data[2:4].send(ify)
target_ify = target[2:4].send(alice)

In [0]:
datasets = [(data_bob, target_bob), (data_ify, target_alice)]

In [0]:
def train(iterations=20):

    model = nn.Linear(2,1)
    opt = optim.SGD(params=model.parameters(), lr=0.1)
    
    for iter in range(iterations):

        for _data, _target in datasets:

            # send model to the data
            model = model.send(_data.location)

            # do normal training
            opt.zero_grad()
            pred = model(_data)
            loss = ((pred - _target)**2).sum()
            loss.backward()
            opt.step()

            # get smarter model back
            model = model.get()

            print(loss.get())

In [0]:
train()

# **Lesson: Advanced Remote Execution Tools**

In the last section we trained a toy model using Federated Learning. We did this by calling .send() and .get() on our model, sending it to the location of training data, updating it, and then bringing it back. However, at the end of the example we realized that we needed to go a bit further to protect people privacy. Namely, we want to average the gradients BEFORE calling .get(). That way, we won't ever see anyone's exact gradient (thus better protecting their privacy!!!)

But, in order to do this, we need a few more pieces:
use a pointer to send a Tensor directly to another worker
And in addition, while we're here, we're going to learn about a few more advanced tensor operations as well which will help us both with this example and a few in the future!

In [0]:
chidi.clear_objects()
ify.clear_objects()